In [2]:
from dotenv import load_dotenv
load_dotenv()
import os
import google.generativeai as genai
from pinecone import Pinecone, ServerlessSpec

C:\Users\jonha\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# pinecone index is basically a mini db in the pinecone instance
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag", dimension=768, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '44d1787a24827f1a74dbe9f6d57730f7', 'Date': 'Tue, 27 Aug 2024 01:55:04 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [4]:
import json
data = json.load(open("reviews.json"))
data['reviews']

[{'name': 'Andrew Lopez',
  'subject': 'Engineering',
  'stars': 2,
  'review': "Professor Lopez's lectures are hard to follow, and he often rushes through the material. His exams are difficult and he doesn't offer much help outside of class."},
 {'name': 'Ashley Moore',
  'subject': 'Art History',
  'stars': 5,
  'review': 'Professor Moore is passionate about art and her enthusiasm is contagious. Her exams are straightforward if you attend lectures and take good notes.'},
 {'name': 'Christopher Taylor',
  'subject': 'Political Science',
  'stars': 2,
  'review': "Dr. Taylor's lectures are disorganized and it's hard to follow his train of thought. His grading is arbitrary, and he doesn't offer much help outside of class."},
 {'name': 'Daniel Thomas',
  'subject': 'Music',
  'stars': 3,
  'review': 'Professor Thomas is knowledgeable, but his teaching style can be a bit dry. His exams are fair, but you need to stay on top of the material.'},
 {'name': 'David Brown',
  'subject': 'Economi

In [5]:
processed_data = []
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))
model = genai.GenerativeModel("text-embedding-004")
for review in data['reviews']:
    response = genai.embed_content(model="models/text-embedding-004", content=review['review'])
    embedding = response['embedding']
    processed_data.append({
        "values" : embedding,
        "id" : review['name'],
        "metadata" : {
            "subject" : review['subject'],
            "stars" : review['stars'],
            "review" : review['review']
        }
    })
# gemini time

In [11]:
index = pc.Index('rag')
index.upsert(
    vectors=processed_data,
    namespace='ns1'
)

{'upserted_count': 20}